In [72]:
import numpy as np
import pandas as pd
import random
import itertools

# Define rating history
user_ratings = [
    [0, 3, 0, 5, 0, 0, 4, 5, 0, 2],  # User 0
    [0, 0, 3, 2, 5, 0, 4, 0, 3, 0],  # User 1
    [3, 1, 1, 3, 5, 5, 0, 4, 0, 0],  # User 2
    [0, 5, 0, 0, 0, 5, 1, 5, 5, 2],  # User 3
    [0, 0, 0, 5, 0, 5, 0, 0, 1, 0],  # User 4
    [2, 0, 0, 4, 5, 5, 5, 0, 0, 0],  # User 5
    [0, 3, 0, 5, 0, 0, 4, 5, 0, 2],  # User 0
    [0, 0, 3, 2, 5, 0, 4, 0, 3, 0],  # User 1
    [3, 1, 1, 0, 5, 5, 0, 4, 0, 0],  # User 2
    [0, 5, 0, 0, 0, 5, 1, 5, 5, 2],  # User 3
    [0, 0, 0, 5, 0, 5, 0, 0, 1, 0],  # User 4
    [2, 0, 0, 4, 5, 5, 5, 0, 0, 0],  # User 5
    [0, 3, 0, 5, 0, 3, 4, 5, 0, 2],  # User 0
    [0, 0, 3, 5, 5, 0, 4, 0, 3, 0],  # User 1
    [3, 1, 1, 3, 5, 5, 0, 4, 0, 0],  # User 2
    [0, 5, 0, 0, 0, 5, 1, 5, 5, 2],  # User 3
    [0, 0, 0, 5, 0, 5, 0, 0, 1, 0],  # User 4
    [2, 0, 0, 4, 5, 2, 5, 0, 0, 0],  # User 5
    [0, 3, 0, 5, 0, 0, 4, 5, 0, 2],  # User 0
    [0, 0, 3, 4, 5, 0, 4, 0, 3, 0],  # User 1
    [3, 1, 1, 3, 5, 5, 0, 4, 0, 0],  # User 2
    [0, 5, 0, 0, 0, 1, 1, 5, 5, 2],  # User 3
    [0, 0, 0, 5, 0, 5, 0, 0, 1, 0],  # User 4
    [2, 0, 0, 3, 5, 0, 5, 0, 0, 0],  # User 5
]

# Input ratings for a new user (0 represents missing values)
user_ratings_input = [0, 5, 3, 2, 0, 0, 1, 0, 0, 2]

# List of books (for reference)
books = ['Book 0', 'Book 1', 'Book 2', 'Book 3', 'Book 4', 'Book 5', 'Book 6', 'Book 7', 'Book 8', 'Book 9']

# Semantic Features
genres = ['action', 'comedy', 'adventure', 'sci-fi', 'crime', 'horror', 'war']
actors = ['Author 1', 'Author 2', 'Author 3']

# Semantic matrix
semantic = np.array([
    [0, 1, 1, 0, 1, 1, 1, 1, 0, 1],
    [0, 0, 1, 1, 1, 0, 0, 1, 0, 1],
    [0, 1, 0, 1, 1, 0, 1, 1, 1, 1],
    [1, 0, 0, 1, 1, 0, 0, 0, 1, 0],
    [0, 1, 0, 0, 0, 1, 0, 0, 1, 1],
    [1, 0, 1, 0, 1, 0, 1, 0, 1, 1],
    [0, 1, 0, 1, 0, 1, 1, 1, 1, 0],
    [0, 1, 0, 1, 1, 0, 0, 1, 0, 1],
    [1, 1, 1, 0, 0, 1, 1, 1, 0, 0],
    [1, 1, 1, 1, 0, 0, 0, 1, 0, 0]
])

In [75]:
import random
import numpy as np
random.seed(42)

import random
import numpy as np
import itertools

# Constants
POPULATION_SIZE = 50
MUTATION_RATE = 0.1
NUM_GENERATIONS = 6
no_rec = 3

def calculate_semrating(solution):
    semrating = 0
    for i in range(len(solution)):
        for j in range(i + 1, len(solution)):
            semrating += semantic[solution[i]][solution[j]]
    return semrating

def possible_solutions(user_ratings_input, no_rec):
    missing_items = [index for index, rating in enumerate(user_ratings_input) if rating == 0]
    all_recommendations = list(itertools.combinations(missing_items, no_rec))
    return all_recommendations

def generate_initial_population(all_recommendations, semratings):
    population = []
    for _ in range(POPULATION_SIZE):
        selected_solution = roulette_wheel_selection(all_recommendations, semratings)
        population.append(selected_solution)
    return population

def roulette_wheel_selection(population, semratings):
    fitness_sum = sum(semratings)
    probabilities = [fitness_score / fitness_sum for fitness_score in semratings]
    cumulative_probabilities = [sum(probabilities[:i+1]) for i in range(len(probabilities))]

    random_number = random.random()
    for i, cumulative_prob in enumerate(cumulative_probabilities):
        if random_number <= cumulative_prob:
            return population[i]

def calculate_total_rating(solution, user_ratings):
    total_predicted_rating = 0
    for item_index in solution:
        target_user_ratings = user_ratings[item_index]
        predicted_rating = calculate_predicted_rating_aws(target_user_ratings, item_index, user_ratings)
        total_predicted_rating += predicted_rating
    return total_predicted_rating

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def average_rating(user_ratings):
    ratings = [rating for rating in user_ratings if rating != 0]
    if len(ratings) > 0:
        return np.mean(ratings)
    else:
        return 0


def calculate_predicted_rating_aws(target_user_ratings, target_item_index, user_ratings):
    average_rating_target_user = average_rating(target_user_ratings)
    total_weighted_rating = 0
    total_similarity = 0

    for user_ratings in user_ratings:
        if user_ratings[target_item_index] != 0:
            similarity = cosine_similarity(target_user_ratings, user_ratings)
            average_rating_user_j = average_rating(user_ratings)
            adjusted_rating = user_ratings[target_item_index] - average_rating_user_j
            total_weighted_rating += similarity * adjusted_rating
            total_similarity += abs(similarity)

    if total_similarity != 0:
        predicted_rating = average_rating_target_user + (total_weighted_rating / total_similarity)
    else:
        predicted_rating = average_rating_target_user

    return predicted_rating

def selection(population, fitness_scores):
    selected_population = []
    total_fitness = sum(fitness_scores)
    probabilities = [fitness_score / total_fitness for fitness_score in fitness_scores]
    for _ in range(POPULATION_SIZE):
        selected_solution = random.choices(population, probabilities)[0]
        selected_population.append(selected_solution)
    return selected_population

def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = list(parent1[:crossover_point]) + list(parent2[crossover_point:])
    child2 = list(parent2[:crossover_point]) + list(parent1[crossover_point:])

    child1 = remove_duplicates(child1)
    child2 = remove_duplicates(child2)

    return child1, child2

def mutation(solution, all_recommendations):
    mutated_solution = solution.copy()
    for i in range(len(solution)):
        if random.random() < MUTATION_RATE:
            possible_items = all_recommendations[i % len(all_recommendations)]
            mutated_solution[i] = random.choice(possible_items)

    mutated_solution = remove_duplicates(mutated_solution)
    return mutated_solution

def remove_duplicates(solution):
    seen = set()
    return [x for x in solution if not (x in seen or seen.add(x))]

def genetic_algorithm(user_ratings_input, no_rec):
    all_recommendations = possible_solutions(user_ratings_input, no_rec)
    semratings = [calculate_semrating(solution) for solution in all_recommendations]
    population = generate_initial_population(all_recommendations, semratings)

    for generation in range(NUM_GENERATIONS):
        fitness_scores = [calculate_total_rating(solution, user_ratings_input) for solution in population]
        selected_population = selection(population, fitness_scores)

        next_population = []
        for _ in range(POPULATION_SIZE // 2):
            parent1, parent2 = random.sample(selected_population, 2)
            child1, child2 = crossover(parent1, parent2)
            mutated_child1 = mutation(child1, all_recommendations)
            mutated_child2 = mutation(child2, all_recommendations)
            next_population.extend([mutated_child1, mutated_child2])

        population = next_population

    best_solution = max(population, key=lambda x: (calculate_total_rating(x, user_ratings_input), calculate_semrating(x)))
    return best_solution

# Input ratings for a new user (0 represents missing values)
user_ratings_input = [0, 5, 3, 2, 0, 0, 1, 0, 0, 2]

best_solution = genetic_algorithm(user_ratings_input, no_rec)
recommendations = [books[i] for i in best_solution]

print("Best solution:", recommendations)


TypeError: 'int' object is not iterable

In [76]:
# Run the genetic algorithm
no_rec = 4  # Number of items in each recommendation list
best_solution = genetic_algorithm(user_ratings_input, no_rec)

# Print the best solution
recommendations = [books[i] for i in best_solution]
print("Best Solution:")
print(recommendations)

TypeError: 'int' object is not iterable

### Final đi mà

In [ ]:
import numpy as np
import pandas as pd
import random
import itertools

# Define rating history
user_ratings = [
    [0, 3, 0, 5, 0, 0, 4, 5, 0, 2],  # User 0
    [0, 0, 3, 2, 5, 0, 4, 0, 3, 0],  # User 1
    [3, 1, 1, 3, 5, 5, 0, 4, 0, 0],  # User 2
    [0, 5, 0, 0, 0, 5, 1, 5, 5, 2],  # User 3
    [0, 0, 0, 5, 0, 5, 0, 0, 1, 0],  # User 4
    [2, 0, 0, 4, 5, 5, 5, 0, 0, 0],  # User 5
    [0, 3, 0, 5, 0, 0, 4, 5, 0, 2],  # User 0
    [0, 0, 3, 2, 5, 0, 4, 0, 3, 0],  # User 1
    [3, 1, 1, 0, 5, 5, 0, 4, 0, 0],  # User 2
    [0, 5, 0, 0, 0, 5, 1, 5, 5, 2],  # User 3
    [0, 0, 0, 5, 0, 5, 0, 0, 1, 0],  # User 4
    [2, 0, 0, 4, 5, 5, 5, 0, 0, 0],  # User 5
    [0, 3, 0, 5, 0, 3, 4, 5, 0, 2],  # User 0
    [0, 0, 3, 5, 5, 0, 4, 0, 3, 0],  # User 1
    [3, 1, 1, 3, 5, 5, 0, 4, 0, 0],  # User 2
    [0, 5, 0, 0, 0, 5, 1, 5, 5, 2],  # User 3
    [0, 0, 0, 5, 0, 5, 0, 0, 1, 0],  # User 4
    [2, 0, 0, 4, 5, 2, 5, 0, 0, 0],  # User 5
    [0, 3, 0, 5, 0, 0, 4, 5, 0, 2],  # User 0
    [0, 0, 3, 4, 5, 0, 4, 0, 3, 0],  # User 1
    [3, 1, 1, 3, 5, 5, 0, 4, 0, 0],  # User 2
    [0, 5, 0, 0, 0, 1, 1, 5, 5, 2],  # User 3
    [0, 0, 0, 5, 0, 5, 0, 0, 1, 0],  # User 4
    [2, 0, 0, 3, 5, 0, 5, 0, 0, 0],  # User 5
]

# Input ratings for a new user (0 represents missing values)
user_ratings_input = [0, 5, 3, 2, 0, 0, 1, 0, 0, 2]

# List of books (for reference)
books = ['Book 0', 'Book 1', 'Book 2', 'Book 3', 'Book 4', 'Book 5', 'Book 6', 'Book 7', 'Book 8', 'Book 9']

# Semantic Features
genres = ['action', 'comedy', 'adventure', 'sci-fi', 'crime', 'horror', 'war']
actors = ['Author 1', 'Author 2', 'Author 3']

# Semantic matrix
semantic = np.array([
    [0, 1, 1, 0, 1, 1, 1, 1, 0, 1],
    [0, 0, 1, 1, 1, 0, 0, 1, 0, 1],
    [0, 1, 0, 1, 1, 0, 1, 1, 1, 1],
    [1, 0, 0, 1, 1, 0, 0, 0, 1, 0],
    [0, 1, 0, 0, 0, 1, 0, 0, 1, 1],
    [1, 0, 1, 0, 1, 0, 1, 0, 1, 1],
    [0, 1, 0, 1, 0, 1, 1, 1, 1, 0],
    [0, 1, 0, 1, 1, 0, 0, 1, 0, 1],
    [1, 1, 1, 0, 0, 1, 1, 1, 0, 0],
    [1, 1, 1, 1, 0, 0, 0, 1, 0, 0]
])


In [ ]:
# Constants
POPULATION_SIZE = 50
MUTATION_RATE = 0.1
NUM_GENERATIONS = 6
no_rec = 3


def calculate_semrating(solution):
    semrating = 0
    for i in range(len(solution)):
        for j in range(i + 1, len(solution)):
            semrating += semantic[solution[i]][solution[j]]
    return semrating

def possible_solutions(user_ratings_input, no_rec):
    missing_items = [index for index, rating in enumerate(user_ratings_input) if rating == 0]
    all_recommendations = list(itertools.combinations(missing_items, no_rec))
    return all_recommendations

def generate_initial_population(all_recommendations, semratings):
    total_semrating = sum(semratings)
    probabilities = [semrating / total_semrating for semrating in semratings]
    population = random.choices(all_recommendations, probabilities, k=POPULATION_SIZE)
    return population

def calculate_total_rating(solution):
    total_rating = 0
    for i in range(len(solution)):
        total_rating += user_ratings[solution[i]][i]
    return total_rating

def selection(population, fitness_scores):
    selected_population = []
    total_fitness = sum(fitness_scores)
    probabilities = [fitness_score / total_fitness for fitness_score in fitness_scores]
    for _ in range(POPULATION_SIZE):
        selected_solution = random.choices(population, probabilities)[0]
        selected_population.append(selected_solution)
    return selected_population


def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = list(parent1[:crossover_point]) + list(parent2[crossover_point:])
    child2 = list(parent2[:crossover_point]) + list(parent1[crossover_point:])

    child1 = remove_duplicates(child1)
    child2 = remove_duplicates(child2)

    return child1, child2

def mutation(solution, all_recommendations):
    mutated_solution = solution.copy()
    for i in range(len(solution)):
        if random.random() < MUTATION_RATE:
            possible_items = all_recommendations[i % len(all_recommendations)]
            mutated_solution[i] = random.choice(possible_items)

    mutated_solution = remove_duplicates(mutated_solution)

    return mutated_solution


def remove_duplicates(solution):
    seen = set()
    return [x for x in solution if not (x in seen or seen.add(x))]


def genetic_algorithm(user_ratings_input, no_rec):
    all_recommendations = possible_solutions(user_ratings_input, no_rec)
    semratings = [calculate_semrating(solution) for solution in all_recommendations]
    population = generate_initial_population(all_recommendations, semratings)

    for generation in range(NUM_GENERATIONS):
        fitness_scores = [calculate_total_rating(solution) for solution in population]

        selected_population = selection(population, fitness_scores)

        next_population = []
        for _ in range(POPULATION_SIZE // 2):
            parent1, parent2 = random.sample(selected_population, 2)
            child1, child2 = crossover(parent1, parent2)
            mutated_child1 = mutation(child1, all_recommendations)
            mutated_child2 = mutation(child2, all_recommendations)
            next_population.extend([mutated_child1, mutated_child2])

        population = next_population

    best_solution = max(population, key=calculate_total_rating)
    return best_solution


In [ ]:
# Run the genetic algorithm
no_rec = 5  # Number of items in each recommendation list
best_solution = genetic_algorithm(user_ratings_input, no_rec)

# Print the best solution
recommendations = [books[i] for i in best_solution]
print("Best Solution:")
print(recommendations)

### Experiment

In [78]:
import numpy as np
import pandas as pd
import random
import itertools

# Define rating history
user_ratings = [
    [0, 3, 0, 5, 0, 0, 4, 5, 0, 2],  # User 0
    [0, 0, 3, 2, 5, 0, 4, 0, 3, 0],  # User 1
    [3, 1, 1, 3, 5, 5, 0, 4, 0, 0],  # User 2
    [0, 5, 0, 0, 0, 5, 1, 5, 5, 2],  # User 3
    [0, 0, 0, 5, 0, 5, 0, 0, 1, 0],  # User 4
    [2, 0, 0, 4, 5, 5, 5, 0, 0, 0],  # User 5
    [0, 3, 0, 5, 0, 0, 4, 5, 0, 2],  # User 0
    [0, 0, 3, 2, 5, 0, 4, 0, 3, 0],  # User 1
    [3, 1, 1, 0, 5, 5, 0, 4, 0, 0],  # User 2
    [0, 5, 0, 0, 0, 5, 1, 5, 5, 2],  # User 3
    [0, 0, 0, 5, 0, 5, 0, 0, 1, 0],  # User 4
    [2, 0, 0, 4, 5, 5, 5, 0, 0, 0],  # User 5
    [0, 3, 0, 5, 0, 3, 4, 5, 0, 2],  # User 0
    [0, 0, 3, 5, 5, 0, 4, 0, 3, 0],  # User 1
    [3, 1, 1, 3, 5, 5, 0, 4, 0, 0],  # User 2
    [0, 5, 0, 0, 0, 5, 1, 5, 5, 2],  # User 3
    [0, 0, 0, 5, 0, 5, 0, 0, 1, 0],  # User 4
    [2, 0, 0, 4, 5, 2, 5, 0, 0, 0],  # User 5
    [0, 3, 0, 5, 0, 0, 4, 5, 0, 2],  # User 0
    [0, 0, 3, 4, 5, 0, 4, 0, 3, 0],  # User 1
    [3, 1, 1, 3, 5, 5, 0, 4, 0, 0],  # User 2
    [0, 5, 0, 0, 0, 1, 1, 5, 5, 2],  # User 3
    [0, 0, 0, 5, 0, 5, 0, 0, 1, 0],  # User 4
    [2, 0, 0, 3, 5, 0, 5, 0, 0, 0],  # User 5
]

# Input ratings for a new user (0 represents missing values)
user_ratings_input = [0, 5, 3, 2, 0, 0, 1, 0, 0, 2]

# List of books (for reference)
books = ['Book 0', 'Book 1', 'Book 2', 'Book 3', 'Book 4', 'Book 5', 'Book 6', 'Book 7', 'Book 8', 'Book 9']

# Semantic Features
genres = ['action', 'comedy', 'adventure', 'sci-fi', 'crime', 'horror', 'war']
actors = ['Author 1', 'Author 2', 'Author 3']

# Semantic matrix
semantic = np.array([
    [0, 1, 1, 0, 1, 1, 1, 1, 0, 1],
    [0, 0, 1, 1, 1, 0, 0, 1, 0, 1],
    [0, 1, 0, 1, 1, 0, 1, 1, 1, 1],
    [1, 0, 0, 1, 1, 0, 0, 0, 1, 0],
    [0, 1, 0, 0, 0, 1, 0, 0, 1, 1],
    [1, 0, 1, 0, 1, 0, 1, 0, 1, 1],
    [0, 1, 0, 1, 0, 1, 1, 1, 1, 0],
    [0, 1, 0, 1, 1, 0, 0, 1, 0, 1],
    [1, 1, 1, 0, 0, 1, 1, 1, 0, 0],
    [1, 1, 1, 1, 0, 0, 0, 1, 0, 0]
])


In [109]:
import random
random.seed(42)  # Set the random seed to 42 or any other desired value


# Constants
POPULATION_SIZE = 50
MUTATION_RATE = 0.1
NUM_GENERATIONS = 6
no_rec = 3


def calculate_semrating(solution):
    semrating = 0
    for i in range(len(solution)):
        for j in range(i + 1, len(solution)):
            semrating += semantic[solution[i]][solution[j]]
    return semrating

def possible_solutions(user_ratings_input, no_rec):
    missing_items = [index for index, rating in enumerate(user_ratings_input) if rating == 0]
    all_recommendations = list(itertools.combinations(missing_items, no_rec))
    return all_recommendations

def generate_initial_population(all_recommendations, semratings):
    population = []
    for _ in range(POPULATION_SIZE):
        selected_solution = roulette_wheel_selection(all_recommendations, semratings)
        population.append(selected_solution)
    return population

def roulette_wheel_selection(population, semratings):
    fitness_sum = sum(semratings)
    probabilities = [fitness_score / fitness_sum for fitness_score in semratings]
    cumulative_probabilities = [sum(probabilities[:i+1]) for i in range(len(probabilities))]

    random_number = random.random()
    for i, cumulative_prob in enumerate(cumulative_probabilities):
        if random_number <= cumulative_prob:
            return population[i]

def calculate_total_rating(solution):
    total_rating = 0
    for i, book_index in enumerate(solution):
        total_rating += user_ratings[i][book_index]
    return total_rating


def selection(population, fitness_scores):
    selected_population = []
    total_fitness = sum(fitness_scores)
    probabilities = [fitness_score / total_fitness for fitness_score in fitness_scores]
    for _ in range(POPULATION_SIZE):
        selected_solution = random.choices(population, probabilities)[0]
        selected_population.append(selected_solution)
    return selected_population


def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = list(parent1[:crossover_point]) + list(parent2[crossover_point:])
    child2 = list(parent2[:crossover_point]) + list(parent1[crossover_point:])

    child1 = remove_duplicates(child1)
    child2 = remove_duplicates(child2)

    return child1, child2

# ensure that the output always contains the specified number of items (no_rec)
def mutation(solution, all_recommendations):
    mutated_solution = solution.copy()
    for i in range(len(solution)):
        if random.random() < MUTATION_RATE:
            possible_items = all_recommendations[i % len(all_recommendations)]
            mutated_solution[i] = random.choice(possible_items)

    mutated_solution = remove_duplicates(mutated_solution)

    return mutated_solution


def genetic_algorithm(user_ratings_input, no_rec):
    all_recommendations = possible_solutions(user_ratings_input, no_rec)
    semratings = [calculate_semrating(solution) for solution in all_recommendations]
    population = generate_initial_population(all_recommendations, semratings)

    for generation in range(NUM_GENERATIONS):
        fitness_scores = [calculate_total_rating(solution) for solution in population]

        selected_population = selection(population, fitness_scores)

        next_population = []
        for _ in range(POPULATION_SIZE // 2):
            parent1, parent2 = random.sample(selected_population, 2)
            child1, child2 = crossover(parent1, parent2)
            mutated_child1 = mutation(child1, all_recommendations)
            mutated_child2 = mutation(child2, all_recommendations)
            next_population.extend([mutated_child1, mutated_child2])

        population = next_population

    best_solution = max(population, key=calculate_total_rating)
    return best_solution


# Run the genetic algorithm
no_rec = 5  # Number of items in each recommendation list
best_solution = genetic_algorithm(user_ratings_input, no_rec)

# Print the best solution
recommendations = [books[i] for i in best_solution]
print("Best Solution:")
print(recommendations)



Best Solution:
['Book 0', 'Book 4', 'Book 5', 'Book 7']


In [117]:
import random
random.seed(42)  # Set the random seed to 42 or any other desired value


# Constants
POPULATION_SIZE = 50
MUTATION_RATE = 0.1
NUM_GENERATIONS = 6
no_rec = 3


def calculate_semrating(solution):
    semrating = 0
    for i in range(len(solution)):
        for j in range(i + 1, len(solution)):
            semrating += semantic[solution[i]][solution[j]]
    return semrating

def possible_solutions(user_ratings_input, no_rec):
    missing_items = [index for index, rating in enumerate(user_ratings_input) if rating == 0]
    all_recommendations = list(itertools.combinations(missing_items, no_rec))
    return all_recommendations

def generate_initial_population(all_recommendations, semratings):
    population = []
    for _ in range(POPULATION_SIZE):
        selected_solution = roulette_wheel_selection(all_recommendations, semratings)
        population.append(selected_solution)
    return population

def roulette_wheel_selection(population, semratings):
    fitness_sum = sum(semratings)
    probabilities = [fitness_score / fitness_sum for fitness_score in semratings]
    cumulative_probabilities = [sum(probabilities[:i+1]) for i in range(len(probabilities))]

    random_number = random.random()
    for i, cumulative_prob in enumerate(cumulative_probabilities):
        if random_number <= cumulative_prob:
            return population[i]

def calculate_total_rating(solution, user_ratings):
    total_predicted_rating = 0
    for item_index in solution:
        target_user_ratings = user_ratings[item_index]
        predicted_rating = calculate_predicted_rating_aws(target_user_ratings, item_index, user_ratings)
        total_predicted_rating += predicted_rating
    return total_predicted_rating

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def average_rating(user_ratings):
    ratings = [rating for rating in user_ratings if rating != 0]
    return np.mean(ratings)

def calculate_predicted_rating_aws(target_user_ratings, target_item_index, user_ratings):
    average_rating_target_user = average_rating(target_user_ratings)
    total_weighted_rating = 0
    total_similarity = 0

    for user_ratings in user_ratings:
        if user_ratings[target_item_index] != 0:
            similarity = cosine_similarity(target_user_ratings, user_ratings)
            average_rating_user_j = average_rating(user_ratings)
            adjusted_rating = user_ratings[target_item_index] - average_rating_user_j
            total_weighted_rating += similarity * adjusted_rating
            total_similarity += abs(similarity)

    if total_similarity != 0:
        predicted_rating = average_rating_target_user + (total_weighted_rating / total_similarity)
    else:
        predicted_rating = average_rating_target_user

    return predicted_rating


def selection(population, fitness_scores):
    selected_population = []
    total_fitness = sum(fitness_scores)
    probabilities = [fitness_score / total_fitness for fitness_score in fitness_scores]
    for _ in range(POPULATION_SIZE):
        selected_solution = random.choices(population, probabilities)[0]
        selected_population.append(selected_solution)
    return selected_population


def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = list(parent1[:crossover_point]) + list(parent2[crossover_point:])
    child2 = list(parent2[:crossover_point]) + list(parent1[crossover_point:])

    child1 = remove_duplicates(child1)
    child2 = remove_duplicates(child2)

    return child1, child2

# ensure that the output always contains the specified number of items (no_rec)
def mutation(solution, all_recommendations):
    mutated_solution = solution.copy()
    for i in range(len(solution)):
        if random.random() < MUTATION_RATE:
            possible_items = all_recommendations[i % len(all_recommendations)]
            mutated_solution[i] = random.choice(possible_items)

    mutated_solution = remove_duplicates(mutated_solution)

    return mutated_solution


def genetic_algorithm(user_ratings_input, no_rec):
    all_recommendations = possible_solutions(user_ratings_input, no_rec)
    semratings = [calculate_semrating(solution) for solution in all_recommendations]
    population = generate_initial_population(all_recommendations, semratings)

    for generation in range(NUM_GENERATIONS):
        fitness_scores = [calculate_total_rating(solution, user_ratings) for solution in population]
        best_solution_index = np.argmax(fitness_scores)
        best_solution = population[best_solution_index]

        selected_population = selection(population, fitness_scores)
        next_population = []

        for _ in range(POPULATION_SIZE // 2):
            parent1, parent2 = random.sample(selected_population, 2)
            child1, child2 = crossover(parent1, parent2)
            mutated_child1 = mutation(child1, all_recommendations)
            mutated_child2 = mutation(child2, all_recommendations)
            next_population.extend([mutated_child1, mutated_child2])

        population = next_population
        population.append(best_solution)  # Include the best solution in the next population

    best_solution = max(population, key=lambda solution: calculate_total_rating(solution, user_ratings))
    return best_solution


# Run the genetic algorithm
no_rec = 4  # Number of items in each recommendation list
best_solution = genetic_algorithm(user_ratings_input, no_rec)

# Print the best solution
recommendations = [books[i] for i in best_solution]
print("Best Solution:")
print(recommendations)



Best Solution:
['Book 0', 'Book 4', 'Book 5', 'Book 7']


In [153]:
import random
random.seed(42)  # Set the random seed to 42 or any other desired value


# Constants
POPULATION_SIZE = 50
MUTATION_RATE = 0.1
NUM_GENERATIONS = 6
no_rec = 3


def calculate_semrating(solution):
    semrating = 0
    for i in range(len(solution)):
        for j in range(i + 1, len(solution)):
            semrating += semantic[solution[i]][solution[j]]
    return semrating

def possible_solutions(user_ratings_input, no_rec):
    missing_items = [index for index, rating in enumerate(user_ratings_input) if rating == 0]
    all_recommendations = list(itertools.combinations(missing_items, no_rec))
    return all_recommendations

def generate_initial_population(all_recommendations, semratings):
    population = []
    for _ in range(POPULATION_SIZE):
        selected_solution = roulette_wheel_selection(all_recommendations, semratings)
        population.append(selected_solution)
    return population

def roulette_wheel_selection(population, semratings):
    fitness_sum = sum(semratings)
    probabilities = [fitness_score / fitness_sum for fitness_score in semratings]
    cumulative_probabilities = [sum(probabilities[:i+1]) for i in range(len(probabilities))]

    random_number = random.random()
    for i, cumulative_prob in enumerate(cumulative_probabilities):
        if random_number <= cumulative_prob:
            return population[i]

def calculate_total_rating(solution, user_ratings):
    total_predicted_rating = 0
    for item_index in solution:
        target_user_ratings = user_ratings[item_index]
        predicted_rating = calculate_predicted_rating_aws(target_user_ratings, item_index, user_ratings)
        total_predicted_rating += predicted_rating
    return total_predicted_rating

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def average_rating(user_ratings):
    ratings = [rating for rating in user_ratings if rating != 0]
    return np.mean(ratings)

def calculate_predicted_rating_aws(target_user_ratings, target_item_index, user_ratings):
    average_rating_target_user = average_rating(target_user_ratings)
    total_weighted_rating = 0
    total_similarity = 0

    for user_ratings in user_ratings:
        if user_ratings[target_item_index] != 0:
            similarity = cosine_similarity(target_user_ratings, user_ratings)
            average_rating_user_j = average_rating(user_ratings)
            adjusted_rating = user_ratings[target_item_index] - average_rating_user_j
            total_weighted_rating += similarity * adjusted_rating
            total_similarity += abs(similarity)

    if total_similarity != 0:
        predicted_rating = average_rating_target_user + (total_weighted_rating / total_similarity)
    else:
        predicted_rating = average_rating_target_user

    return predicted_rating


def selection(population, fitness_scores):
    selected_population = []
    total_fitness = sum(fitness_scores)
    probabilities = [fitness_score / total_fitness for fitness_score in fitness_scores]
    for _ in range(POPULATION_SIZE):
        selected_solution = random.choices(population, probabilities)[0]
        selected_population.append(selected_solution)
    return selected_population


def crossover(parent1, parent2):
    if len(parent1) <= 1:
        return parent1, parent2

    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = list(parent1[:crossover_point]) + list(parent2[crossover_point:])
    child2 = list(parent2[:crossover_point]) + list(parent1[crossover_point:])

    child1 = remove_duplicates(child1)
    child2 = remove_duplicates(child2)

    return child1, child2


# ensure that the output always contains the specified number of items (no_rec)
def mutation(solution, all_recommendations):
    mutated_solution = list(solution)
    for i in range(len(solution)):
        if random.random() < MUTATION_RATE:
            possible_items = all_recommendations[i % len(all_recommendations)]
            mutated_solution[i] = random.choice(possible_items)

    mutated_solution = remove_duplicates(mutated_solution)

    return tuple(mutated_solution)


def genetic_algorithm(user_ratings_input, no_rec):

    num_missing_values = sum(1 for rating in user_ratings_input if rating == 0)

    # Check if no_rec is smaller than the number of missing values
    if no_rec > num_missing_values:
        raise ValueError("The value of no_rec must be smaller than the number of missing values.")

    all_recommendations = possible_solutions(user_ratings_input, no_rec)
    semratings = [calculate_semrating(solution) for solution in all_recommendations]
    population = generate_initial_population(all_recommendations, semratings)

    for generation in range(NUM_GENERATIONS):
        fitness_scores = [calculate_total_rating(solution, user_ratings) for solution in population]
        best_solution_index = np.argmax(fitness_scores)
        best_solution = population[best_solution_index]

        selected_population = selection(population, fitness_scores)
        next_population = []

        for _ in range(POPULATION_SIZE // 2):
            parent1, parent2 = random.sample(selected_population, 2)
            child1, child2 = crossover(parent1, parent2)
            mutated_child1 = mutation(child1, all_recommendations)
            mutated_child2 = mutation(child2, all_recommendations)
            next_population.extend([mutated_child1, mutated_child2])

        population = next_population
        population.append(best_solution)  # Include the best solution in the next population

    best_solution = max(population, key=lambda solution: calculate_total_rating(solution, user_ratings))
    return best_solution


# Run the genetic algorithm
no_rec = 5  # Number of items in each recommendation list
best_solution = genetic_algorithm(user_ratings_input, no_rec)

# Print the best solution
recommendations = [books[i] for i in best_solution]
print("Best Solution:")
print(recommendations)



Best Solution:
['Book 0', 'Book 4', 'Book 5', 'Book 7', 'Book 8']


In [163]:
import random
import itertools
import numpy as np

random.seed(42)  # Set the random seed to 42 or any other desired value

# Constants
no_rec = 3

def calculate_semrating(solution):
    semrating = 0
    for i in range(len(solution)):
        for j in range(i + 1, len(solution)):
            semrating += semantic[solution[i]][solution[j]]
    return semrating

def possible_solutions(user_ratings_input, no_rec):
    missing_items = [index for index, rating in enumerate(user_ratings_input) if rating == 0]
    all_recommendations = list(itertools.combinations(missing_items, no_rec))
    return all_recommendations

def generate_initial_population(all_recommendations, semratings):
    population = []
    for _ in range(population_size):
        selected_solution = roulette_wheel_selection(all_recommendations, semratings)
        population.append(selected_solution)
    return population

def roulette_wheel_selection(population, semratings):
    fitness_sum = sum(semratings)
    probabilities = [fitness_score / fitness_sum for fitness_score in semratings]
    cumulative_probabilities = [sum(probabilities[:i+1]) for i in range(len(probabilities))]

    random_number = random.random()
    for i, cumulative_prob in enumerate(cumulative_probabilities):
        if random_number <= cumulative_prob:
            return population[i]

def calculate_total_rating(solution, user_ratings):
    total_predicted_rating = 0
    for item_index in solution:
        target_user_ratings = user_ratings[item_index]
        predicted_rating = calculate_predicted_rating_aws(target_user_ratings, item_index, user_ratings)
        total_predicted_rating += predicted_rating
    return total_predicted_rating

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def average_rating(user_ratings):
    ratings = [rating for rating in user_ratings if rating != 0]
    return np.mean(ratings)

def calculate_predicted_rating_aws(target_user_ratings, target_item_index, user_ratings):
    average_rating_target_user = average_rating(target_user_ratings)
    total_weighted_rating = 0
    total_similarity = 0

    for user_ratings in user_ratings:
        if user_ratings[target_item_index] != 0:
            similarity = cosine_similarity(target_user_ratings, user_ratings)
            average_rating_user_j = average_rating(user_ratings)
            adjusted_rating = user_ratings[target_item_index] - average_rating_user_j
            total_weighted_rating += similarity * adjusted_rating
            total_similarity += abs(similarity)

    if total_similarity != 0:
        predicted_rating = average_rating_target_user + (total_weighted_rating / total_similarity)
    else:
        predicted_rating = average_rating_target_user

    return predicted_rating


def selection(population, fitness_scores):
    selected_population = []
    total_fitness = sum(fitness_scores)
    probabilities = [fitness_score / total_fitness for fitness_score in fitness_scores]
    for _ in range(population_size):
        selected_solution = random.choices(population, probabilities)[0]
        selected_population.append(selected_solution)
    return selected_population


def crossover(parent1, parent2):
    if len(parent1) <= 1:
        return parent1, parent2

    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = list(parent1[:crossover_point]) + list(parent2[crossover_point:])
    child2 = list(parent2[:crossover_point]) + list(parent1[crossover_point:])

    child1 = remove_duplicates(child1)
    child2 = remove_duplicates(child2)

    return child1, child2


# ensure that the output always contains the specified number of items (no_rec)
def mutation(solution, all_recommendations):
    mutated_solution = list(solution)
    for i in range(len(solution)):
        if random.random() < mutation_rate:
            possible_items = all_recommendations[i % len(all_recommendations)]
            mutated_solution[i] = random.choice(possible_items)

    mutated_solution = remove_duplicates(mutated_solution)

    return tuple(mutated_solution)


def genetic_algorithm(user_ratings_input, no_rec):

    num_missing_values = sum(1 for rating in user_ratings_input if rating == 0)

    # Check if no_rec is smaller than the number of missing values
    if no_rec > num_missing_values:
        raise ValueError("The value of no_rec must be smaller than the number of missing values.")

    all_recommendations = possible_solutions(user_ratings_input, no_rec)
    semratings = [calculate_semrating(solution) for solution in all_recommendations]
    population = generate_initial_population(all_recommendations, semratings)

    for generation in range(num_generations):
        fitness_scores = [calculate_total_rating(solution, user_ratings) for solution in population]
        best_solution_index = np.argmax(fitness_scores)
        best_solution = population[best_solution_index]

        selected_population = selection(population, fitness_scores)
        next_population = []

        for _ in range(population_size // 2):
            parent1, parent2 = random.sample(selected_population, 2)
            child1, child2 = crossover(parent1, parent2)
            mutated_child1 = mutation(child1, all_recommendations)
            mutated_child2 = mutation(child2, all_recommendations)
            next_population.extend([mutated_child1, mutated_child2])

        population = next_population
        population.append(best_solution)  # Include the best solution in the next population

    best_solution = max(population, key=lambda solution: calculate_total_rating(solution, user_ratings))
    return best_solution


# Run the genetic algorithm
population_size = 50  # Size of the population
mutation_rate = 0.1  # Mutation rate
num_generations = 6  # Number of generations

# Run the genetic algorithm
no_rec = 5  # Number of items in each recommendation list
best_solution = genetic_algorithm(user_ratings_input, no_rec)

# Print the best solution
recommendations = [books[i] for i in best_solution]
print("Best Solution:")
print(recommendations)


Best Solution:
['Book 0', 'Book 4', 'Book 5', 'Book 7', 'Book 8']


In [151]:
no_rec = 4
population_size = 40
mutation_rate = 0.3
num_generations = 4
genetic_algorithm(user_ratings_input, no_rec, selection_size, mutation_rate, num_generations)

TypeError: 'int' object is not iterable

In [147]:
# Define the ranges of hyperparameters to explore
no_rec_range = [3, 4, 5]
selection_size_range = [10, 20]
mutation_rate_range = [0.05, 0.1, 0.2]
num_generations_range = [5, 6, 7]
num_iterations = 3

# Perform hyperparameter tuning
tuning_results = hyperparameter_tuning(user_ratings_input, no_rec_range, selection_size_range, mutation_rate_range, num_generations_range, num_iterations)

# Print the tuning results as a table
print("Hyperparameter Tuning Results:")
print("--------------------------------------------------")
print("| No_Rec | Selection Size | Mutation Rate | Generations | Average Score |")
print("--------------------------------------------------")
for result in tuning_results:
    no_rec, selection_size, mutation_rate, num_generations, average_score = result
    print(f"|   {no_rec}   |       {selection_size}       |     {mutation_rate}     |      {num_generations}      |    {average_score:.2f}    |")
print("--------------------------------------------------")

TypeError: 'int' object is not iterable

In [141]:
def hyperparameter_tuning(user_ratings_input, no_rec_range, selection_size_range, mutation_rate_range, num_generations_range, num_iterations):
    tuning_results = []
    
    for no_rec in no_rec_range:
        for selection_size in selection_size_range:
            for mutation_rate in mutation_rate_range:
                for num_generations in num_generations_range:
                    total_scores = 0

                    for _ in range(num_iterations):
                        try:
                            best_solution = genetic_algorithm(user_ratings_input, no_rec, selection_size, mutation_rate, num_generations)
                            total_scores += calculate_total_rating(best_solution, user_ratings_input)
                        except ValueError:
                            continue

                    average_score = total_scores / num_iterations
                    tuning_results.append((no_rec, selection_size, mutation_rate, num_generations, average_score))

    return tuning_results


### Không phải final

In [ ]:
import random
import numpy as np
random.seed(42)

# Constants
POPULATION_SIZE = 50
MUTATION_RATE = 0.1
NUM_GENERATIONS = 6
no_rec = 3


def calculate_semrating(solution):
    semrating = 0
    for i in range(len(solution)):
        for j in range(i + 1, len(solution)):
            semrating += semantic[solution[i]][solution[j]]
    return semrating

def possible_solutions(user_ratings_input, no_rec):
    missing_items = [index for index, rating in enumerate(user_ratings_input) if rating == 0]
    all_recommendations = list(itertools.combinations(missing_items, no_rec))
    return all_recommendations

# The roulette wheel choose some solutions as the initial population
def generate_initial_population(all_recommendations, semratings):
    population = []
    for _ in range(POPULATION_SIZE):
        selected_solution = roulette_wheel_selection(all_recommendations, semratings)
        population.append(selected_solution)
    return population

def roulette_wheel_selection(population, semratings):
    fitness_sum = sum(semratings)
    probabilities = [fitness_score / fitness_sum for fitness_score in semratings]
    cumulative_probabilities = [sum(probabilities[:i+1]) for i in range(len(probabilities))]

    random_number = random.random()
    for i, cumulative_prob in enumerate(cumulative_probabilities):
        if random_number <= cumulative_prob:
            return population[i]

def calculate_total_rating(solution, user_ratings):
    total_rating = 0
    for item_index in solution:
        total_rating += calculate_predicted_rating_aws(user_ratings[item_index], item_index, user_ratings)
    return total_rating

def average_rating(user_ratings):
    ratings = [rating for rating in user_ratings if isinstance(rating, int) and rating != 0]
    return np.mean(ratings)


def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

import numpy as np

def calculate_total_rating(solution, user_ratings):
    total_predicted_rating = 0
    for item_index in solution:
        target_user_ratings = user_ratings[item_index]
        predicted_rating = calculate_predicted_rating_aws(target_user_ratings, item_index, user_ratings)
        total_predicted_rating += predicted_rating
    return total_predicted_rating

def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def average_rating(user_ratings):
    ratings = [rating for rating in user_ratings if rating != 0]
    return np.mean(ratings)

def calculate_predicted_rating_aws(target_user_ratings, target_item_index, user_ratings):
    average_rating_target_user = average_rating(target_user_ratings)
    total_weighted_rating = 0
    total_similarity = 0

    for user_ratings in user_ratings:
        if user_ratings[target_item_index] != 0:
            similarity = cosine_similarity(target_user_ratings, user_ratings)
            average_rating_user_j = average_rating(user_ratings)
            adjusted_rating = user_ratings[target_item_index] - average_rating_user_j
            total_weighted_rating += similarity * adjusted_rating
            total_similarity += abs(similarity)

    if total_similarity != 0:
        predicted_rating = average_rating_target_user + (total_weighted_rating / total_similarity)
    else:
        predicted_rating = average_rating_target_user

    return predicted_rating

def selection(population, fitness_scores):
    selected_population = []
    total_fitness = sum(fitness_scores)
    probabilities = [fitness_score / total_fitness for fitness_score in fitness_scores]
    for _ in range(POPULATION_SIZE):
        selected_solution = random.choices(population, probabilities)[0]
        selected_population.append(selected_solution)
    return selected_population

def crossover(parent1, parent2):
    if len(parent1) <= 1:
        return parent1, parent2

    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = list(parent1[:crossover_point]) + list(parent2[crossover_point:])
    child2 = list(parent2[:crossover_point]) + list(parent1[crossover_point:])

    child1 = remove_duplicates(child1)
    child2 = remove_duplicates(child2)

    return child1, child2


def mutation(solution, all_recommendations):
    mutated_solution = solution.copy()
    for i in range(len(solution)):
        if random.random() < MUTATION_RATE:
            possible_items = all_recommendations[i % len(all_recommendations)]
            mutated_solution[i] = random.choice(possible_items)

    mutated_solution = remove_duplicates(mutated_solution)
    return mutated_solution

def remove_duplicates(solution):
    seen = set()
    return [x for x in solution if not (x in seen or seen.add(x))]


def genetic_algorithm(user_ratings_input, no_rec):
    all_recommendations = possible_solutions(user_ratings_input, no_rec)
    semratings = [calculate_semrating(solution) for solution in all_recommendations]
    population = generate_initial_population(all_recommendations, semratings)
    
    best_solution = None
    best_fitness = float('-inf')
    highest_semrating = float('-inf')
    
    for generation in range(NUM_GENERATIONS):
        fitness_scores = [calculate_total_rating(solution, user_ratings_input) for solution in population]

        selected_population = selection(population, fitness_scores)

        next_population = []
        for _ in range(POPULATION_SIZE // 2):
            parent1, parent2 = random.sample(selected_population, 2)
            child1, child2 = crossover(parent1, parent2)
            mutated_child1 = mutation(child1, all_recommendations)
            mutated_child2 = mutation(child2, all_recommendations)
            next_population.extend([mutated_child1, mutated_child2])

        population = next_population

        # Update the best solution based on fitness and highest semantic rating
        for solution in population:
            fitness = calculate_total_rating(solution, user_ratings_input)
            semrating = calculate_semrating(solution)
            if fitness > best_fitness or (fitness == best_fitness and semrating > highest_semrating):
                best_solution = solution
                best_fitness = fitness
                highest_semrating = semrating

    return best_solution
